# [Reference](https://tensorflow.google.cn/get_started/estimator#construct_a_deep_neural_network_classifier)

In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import os
from urllib.request import urlopen

import tensorflow as tf
import numpy as np

# Preparation

## Load iris data

In [2]:
IRIS_TRAINING = "iris_training.csv"
IRIS_TRAINING_URL = "http://download.tensorflow.org/data/iris_training.csv"

IRIS_TEST = "iris_test.csv"
IRIS_TEST_URL = "http://download.tensorflow.org/data/iris_test.csv"

# if not os.path.exists(IRIS_TRAINING):
raw = urlopen(IRIS_TRAINING_URL).read().decode()
with open(IRIS_TRAINING,'w') as f:
    f.write(raw)

# if not os.path.exists(IRIS_TEST):
raw = urlopen(IRIS_TEST_URL).read().decode()
with open(IRIS_TEST,'w') as f:
    f.write(raw)

In [3]:
# Load datasets.
train_set = tf.contrib.learn.datasets.base.load_csv_with_header(
    filename=IRIS_TRAINING,
    target_dtype=np.int,
    features_dtype=np.float32)
test_set = tf.contrib.learn.datasets.base.load_csv_with_header(
    filename=IRIS_TEST,
    target_dtype=np.int,
    features_dtype=np.float32)

In [4]:
type(train_set),type(test_set)

(tensorflow.contrib.learn.python.learn.datasets.base.Dataset,
 tensorflow.contrib.learn.python.learn.datasets.base.Dataset)

In [17]:
type(train_set.data), train_set.data.shape

(numpy.ndarray, (120, 4))

## Construct a Deep Neural Network Classifier

tf.estimator offers a variety of predefined models, called Estimators, which you can use "out of the box" to run training and evaluation operations on your data. Here, you'll configure a Deep Neural Network Classifier model to fit the Iris data. 

#### Define a feature column

a feature column define the data type for the data in data set.
there are 3 feature in data, so must set the shape to [4]

In [6]:
# Specify that all features have real-value data
feature_columns = [tf.feature_column.numeric_column("x", shape=[4])]


#### Then use the pre-defined model to fit the data

In [7]:
# Build 3 layer DNN with 10, 20, 10 units respectively.
classifier = tf.estimator.DNNClassifier(feature_columns=feature_columns,
                                        hidden_units=[10, 20, 10],
                                        n_classes=3,
                                        model_dir="/tmp/iris_model")

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_keep_checkpoint_max': 5, '_save_checkpoints_steps': None, '_keep_checkpoint_every_n_hours': 10000, '_model_dir': '/tmp/iris_model', '_save_summary_steps': 100, '_session_config': None, '_tf_random_seed': 1, '_save_checkpoints_secs': 600, '_log_step_count_steps': 100}


## Describe the training input pipeline

The tf.estimator use the input_fn to feed the data. Here we user the tf.estimator.inputs_numpy_input_fn to define the input pipeline

In [12]:
# Define the training inputs
train_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x": np.array(train_set.data)},
    y=np.array(train_set.target),
    num_epochs=None,
    shuffle=True)

## Fit the DNNClassifier to the Iris Training Data

The we can feed the input_fn to the model and start training.

In [13]:
classifier.train(input_fn=train_input_fn, steps=10)

INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 1 into /tmp/iris_model/model.ckpt.
INFO:tensorflow:step = 1, loss = 255.437
INFO:tensorflow:Saving checkpoints for 10 into /tmp/iris_model/model.ckpt.
INFO:tensorflow:Loss for final step: 91.04.


The state of the model is saved in the classifer, so you can continue training later

In [15]:
classifier.train(input_fn=train_input_fn,steps=10)

INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Restoring parameters from /tmp/iris_model/model.ckpt-20
INFO:tensorflow:Saving checkpoints for 21 into /tmp/iris_model/model.ckpt.
INFO:tensorflow:step = 21, loss = 49.3807
INFO:tensorflow:Saving checkpoints for 30 into /tmp/iris_model/model.ckpt.
INFO:tensorflow:Loss for final step: 41.7445.


## Evaluate Model Accuracy

You've trained your DNNClassifier model on the Iris training data; now, you can check its accuracy on the Iris test data using the evaluate method. Like train, evaluate takes an input function that builds its input pipeline. evaluate returns a dicts with the evaluation results. The following code passes the Iris test data—test_set.data and test_set.target—to evaluate and prints the accuracy from the results:

In [24]:
# Define the test inputs
test_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x": np.array(test_set.data)},
    y=np.array(test_set.target),
    num_epochs=1,
    shuffle=False)

# Evaluate accuracy.
accuracy_score = classifier.evaluate(input_fn=test_input_fn)

INFO:tensorflow:Starting evaluation at 2017-09-08-01:28:38
INFO:tensorflow:Restoring parameters from /tmp/iris_model/model.ckpt-2000
INFO:tensorflow:Finished evaluation at 2017-09-08-01:28:38
INFO:tensorflow:Saving dict for global step 2000: accuracy = 0.966667, average_loss = 0.0570463, global_step = 2000, loss = 1.71139


In [25]:
accuracy_score

{'accuracy': 0.96666664,
 'average_loss': 0.057046309,
 'global_step': 2000,
 'loss': 1.7113893}

## Classify New Samples

Use the estimator's predict() method to classify new samples. For example, say you have these two new flower samples:

In [26]:
# Classify two new flower samples.
new_samples = np.array(
    [[6.4, 3.2, 4.5, 1.5],
     [5.8, 3.1, 5.0, 1.7]], dtype=np.float32)
predict_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x": new_samples},
    num_epochs=1,
    shuffle=False)

predictions = list(classifier.predict(input_fn=predict_input_fn))
predicted_classes = [p["classes"] for p in predictions]

print(
    "New Samples, Class Predictions:    {}\n"
    .format(predicted_classes))

INFO:tensorflow:Restoring parameters from /tmp/iris_model/model.ckpt-2000
New Samples, Class Predictions:    [array([b'1'], dtype=object), array([b'2'], dtype=object)]



In [27]:
predictions

[{'class_ids': array([1]),
  'classes': array([b'1'], dtype=object),
  'logits': array([-5.15059233,  4.87604856, -3.81199336], dtype=float32),
  'probabilities': array([  4.41970005e-05,   9.99787271e-01,   1.68553976e-04], dtype=float32)},
 {'class_ids': array([2]),
  'classes': array([b'2'], dtype=object),
  'logits': array([-9.74863625,  0.85466111,  2.27391148], dtype=float32),
  'probabilities': array([  4.83711892e-06,   1.94778189e-01,   8.05216968e-01], dtype=float32)}]